# References
- ["Exploring UPnP with Python" - Electric Monk](https://www.electricmonk.nl/log/2016/07/05/exploring-upnp-with-python/)
- [upnpclient](https://github.com/flyte/upnpclient/blob/develop/upnpclient/soap.py)

#           HTTPU M-Search UDP Broadcast  To UPNP Devices  ----->
# <-----  HTTPU UDP Replies with Location Header

In [310]:
import socket

msg = \
    'M-SEARCH * HTTP/1.1\r\n' \
    'HOST:239.255.255.250:1900\r\n' \
    'ST:upnp:rootdevice\r\n' \
    'MX:2\r\n' \
    'MAN:"ssdp:discover"\r\n' \
    '\r\n'

msg = str.encode(msg)

# Set up UDP socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
s.settimeout(2)
s.sendto(msg, ('239.255.255.250', 1900) )

def get_server_properties(data_str):
    property_dict = {}
    server = ''
    for line in data_str.split('\r\n'):
        if line.lower().startswith('server:'):
            server = line[len('server: '):]
        else:
            k = line.split(': ')[0].lower()
            v = line.split(': ')[-1]

            if k != '' and v != '':
                property_dict[k] = v
                
    return server, property_dict

servers = {}
try:
    while True:
        data, addr = s.recvfrom(65507)
        server, property_dict = get_server_properties(data.decode())
        servers[server] = property_dict
        print(addr)
        print(data.decode())
except socket.timeout:
    pass

('192.168.1.14', 53602)
HTTP/1.1 200 OK
CACHE-CONTROL: max-age=1800
DATE: Sun, 12 Jul 2020 12:49:43 GMT
EXT: 
LOCATION: http://192.168.1.14:7678/nservice/
SERVER: Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0
ST: upnp:rootdevice
USN: uuid:f4ec0a2d-c0aa-4f9d-904a-ecc457202ba9::upnp:rootdevice
Content-Length: 0
BOOTID.UPNP.ORG: 5


('192.168.1.14', 48267)
HTTP/1.1 200 OK
CACHE-CONTROL: max-age=1800
DATE: Sun, 12 Jul 2020 12:49:43 GMT
EXT: 
LOCATION: http://192.168.1.14:9197/dmr
SERVER: Samsung-Linux/4.1, UPnP/1.0, Samsung_UPnP_SDK/1.0
ST: upnp:rootdevice
USN: uuid:036cb031-f5f8-4831-bcd7-c0259ee25740::upnp:rootdevice
Content-Length: 0
BOOTID.UPNP.ORG: 4


('192.168.1.3', 1900)
HTTP/1.1 200 OK
Cache-Control: max-age=3600
ST: upnp:rootdevice
USN: uuid:02706106-8001-10a3-80b0-b0a7374b9165::upnp:rootdevice
Ext: 
Server: Roku/9.3.0 UPnP/1.0 Roku/9.3.0
LOCATION: http://192.168.1.3:8060/
WAKEUP: MAC=b0:a7:37:4b:91:65;Timeout=10




In [320]:
"""
    - search: key word in server name
    - servers: dictionary with key: server names, value: property_dict
    
    returns value from servers dictionary where value=property_dict
"""
def get_server_properties(search, servers):
    assert len(servers) > 0, 'Error: servers dictionary empty.'
    server_key = [k for k in servers.keys() if search.lower() in k.lower()]
    assert len(server_key) > 0, 'Error: No server with specified key word found.'
    assert len(server_key) == 1, 'Server key word not specific enough. More than one server found: ' + ' '.join(server_key)
    server_key = server_key[0]
    
    return servers[server_key]

In [326]:
location_url = get_server_properties('samsung', servers)['location']
print(location_url)

http://192.168.1.14:9197/dmr


# Parse SCPD(Service Control Point Definition) XMLs

## 1. Root XML

In [433]:
from lxml import etree
from io import StringIO, BytesIO
import requests

def xml_pretty_print(obj, tab_cnt=0, filter_txt='', filter_tag='', filter_prt_tag=''):        
    for v in obj.getchildren():
        v_str = v.text.strip()
        t_str = v.tag.split('}')[-1] if '}' in v.tag else v.tag
        if filter_txt.lower() in v_str.lower() and filter_tag.lower() in t_str.lower() and filter_prt_tag.lower() in v.getparent().tag:
            print(''.join(['\t']*tab_cnt)+t_str+'--> '+v_str)
        
        tab_cnt += 1
        tab_cnt = xml_pretty_print(v, tab_cnt=tab_cnt, filter_txt=filter_txt, filter_tag=filter_tag)
    tab_cnt -= 1
        
    return tab_cnt

res = requests.get(location_url)
location_root = etree.fromstring(res.content, base_url=location_url)

In [434]:
xml_pretty_print(location_root)

specVersion--> 
	major--> 1
	minor--> 0
device--> 
	deviceType--> urn:schemas-upnp-org:device:MediaRenderer:1
	X_compatibleId--> MS_DigitalMediaDeviceClass_DMR_V001
	X_deviceCategory--> Display.TV.LCD Multimedia.DMR
	X_DLNADOC--> DMR-1.50
	friendlyName--> [TV] Samsung 6 Series (75)
	manufacturer--> Samsung Electronics
	manufacturerURL--> http://www.samsung.com/sec
	modelDescription--> Samsung TV DMR
	modelName--> UN75NU6950
	modelNumber--> AllShare1.0
	modelURL--> http://www.samsung.com/sec
	serialNumber--> 07ZX3CDN101794W
	UDN--> uuid:036cb031-f5f8-4831-bcd7-c0259ee25740
	iconList--> 
		icon--> 
			mimetype--> image/jpeg
			width--> 48
			height--> 48
			depth--> 24
			url--> /icon_SML.jpg
		icon--> 
			mimetype--> image/jpeg
			width--> 120
			height--> 120
			depth--> 24
			url--> /icon_LRG.jpg
		icon--> 
			mimetype--> image/png
			width--> 48
			height--> 48
			depth--> 24
			url--> /icon_SML.png
		icon--> 
			mimetype--> image/png
			width--> 120
			height--> 120
			depth--> 24
	

-1

In [435]:
results = get_scpd_simple(location_root, filter_tag='urlbase')
if len(results) >= 1:
    soap_req_base_url = results[0].text
else:
    soap_req_base_url = ':'.join(location_url.split(':')[:-1]) + ':' + location_url.split(':')[-1].split('/')[0]
    
print(soap_req_base_url)

http://192.168.1.14:9197


In [751]:
def get_service_urls(location_root, base_url):
    ctrl = get_scpd_simple(location_root, filter_prt_tag='service', filter_tag='controlurl')
    scpd = get_scpd_simple(location_root, filter_prt_tag='service', filter_tag='scpdurl')
    stype = get_scpd_simple(location_root, filter_prt_tag='service', filter_tag='servicetype')
    
    urls = {}
    url_list = []
    for c, s, st in zip(ctrl, scpd, stype):
        url_list += [dict({'ctrl': base_url + c.text, 'scpd': base_url + s.text, 'type': st.text})]
        
    return url_list

urls = get_service_urls(location_root, soap_req_base_url)
urls

[{'ctrl': 'http://192.168.1.14:9197/upnp/control/RenderingControl1',
  'scpd': 'http://192.168.1.14:9197/RenderingControl_1.xml',
  'type': 'urn:schemas-upnp-org:service:RenderingControl:1'},
 {'ctrl': 'http://192.168.1.14:9197/upnp/control/ConnectionManager1',
  'scpd': 'http://192.168.1.14:9197/ConnectionManager_1.xml',
  'type': 'urn:schemas-upnp-org:service:ConnectionManager:1'},
 {'ctrl': 'http://192.168.1.14:9197/upnp/control/AVTransport1',
  'scpd': 'http://192.168.1.14:9197/AVTransport_1.xml',
  'type': 'urn:schemas-upnp-org:service:AVTransport:1'}]

In [788]:
use_idx = 0
ctrl_url = urls[use_idx]['ctrl']
scpd_url = urls[use_idx]['scpd']
service_type = urls[use_idx]['type']
print(ctrl_url)
print(scpd_url)
print(service_type)

http://192.168.1.14:9197/upnp/control/RenderingControl1
http://192.168.1.14:9197/RenderingControl_1.xml
urn:schemas-upnp-org:service:RenderingControl:1


## 2. Service XML

In [789]:
res = requests.get(scpd_url)
scpd_root = etree.fromstring(res.content, base_url=scpd_url)

In [790]:
def get_relatedstatevariable(root, action_name, arg_name):
    action = get_scpd_simple(root, filter_prt_tag='action', filter_tag='name', filter_txt=action_name)
    assert len(action) > 0, '(%s) action not found'%(action_name)
    action = action[0].getparent()
    argument = get_scpd_simple(action, filter_prt_tag='argument', filter_tag='name', filter_txt=arg_name)
    assert len(argument) > 0, '(%s) argument not found'%(arg_name)
    argument = argument[0].getparent()
    return get_scpd_simple(argument, filter_tag='relatedstatevariable')[0].text

def get_statevar_desc(root, action_name, arg_name, show_id=False):
    rsv = get_relatedstatevariable(root, action_name, arg_name)
    statevar = get_scpd_simple(root, filter_prt_tag='statevariable', filter_tag='name', filter_txt=rsv, exact=True)[0].getparent()

    tags = {}
    for sv_tag in statevar.getchildren():
        tg = sv_tag.tag
        if not(show_id) and 'name' in tg:
            continue
        
        if '}' in tg:
            tg = tg.split('}')[-1]
        
        if 'list' in tg.lower():
            arglist = []
            for item in sv_tag.getchildren():
                arglist += [item.text]
            text = ', '.join(arglist)
        else:
            text = sv_tag.text
        tags[tg] = text
        
    return tags

In [791]:
def get_action_arguments(root, action_name):
    action = get_scpd_simple(root, filter_tag='name', filter_prt_tag='action', filter_txt=action_name)
    if len(action) == 0:
        return
    
    action = action[0].getparent()
    return get_scpd_simple(action, filter_prt_tag='argument', filter_tag='name')
        

def pretty_print_actions(actionlist, name_filter='', show_datatype=False): 
    actions = get_scpd_simple(actionlist, filter_tag='action')
    for action in actions:
        name = get_scpd_simple(action, filter_tag='name')[0].text
        if name_filter.lower() not in name.lower():
            continue
            
        arguments = get_scpd_simple(action, filter_tag='argument', filter_prt_tag='argumentlist')
        
        print_str = name + '('
        in_args, out_args = [], []
        for arg in arguments:
            aname = get_scpd_simple(arg, filter_tag='name', filter_prt_tag='argument')[0].text
            adir = get_scpd_simple(arg, filter_tag='direction', filter_prt_tag='argument')[0].text

            desc_dict = get_statevar_desc(scpd_root, name, aname)
            dt = '[%s]'%(desc_dict['dataType']) if 'dataType' in desc_dict and show_datatype else ''
            
            if adir == 'in':
                in_args += [aname + dt]
            elif adir == 'out':
                out_args += [aname + dt]
        print_str += ', '.join(in_args) + ')'
        if len(out_args) > 0:
            print_str += '\r\n\tReturns: (%s)' % (', '.join(out_args)) 
        print(print_str)

In [792]:
xml_pretty_print(scpd_root)

specVersion--> 
	major--> 1
	minor--> 0
actionList--> 
	action--> 
		name--> ListPresets
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> CurrentPresetNameList
				direction--> out
				relatedStateVariable--> PresetNameList
	action--> 
		name--> SelectPreset
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> PresetName
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_PresetName
	action--> 
		name--> GetMute
		argumentList--> 
			argument--> 
				name--> InstanceID
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_InstanceID
			argument--> 
				name--> Channel
				direction--> in
				relatedStateVariable--> A_ARG_TYPE_Channel
			argument--> 
				name--> CurrentMute
				direction--> out
				relatedStateVariable--> Mute
	action--> 
		name--> SetMute
		argumentLi

-1

In [802]:
actionlist = get_scpd_simple(scpd_root, filter_tag='actionlist')[0]
pretty_print_actions(actionlist)

ListPresets(InstanceID)
	Returns: (CurrentPresetNameList)
SelectPreset(InstanceID, PresetName)
GetMute(InstanceID, Channel)
	Returns: (CurrentMute)
SetMute(InstanceID, Channel, DesiredMute)
GetVolume(InstanceID, Channel)
	Returns: (CurrentVolume)
SetVolume(InstanceID, Channel, DesiredVolume)
X_GetAspectRatio(InstanceID)
	Returns: (AspectRatio)
X_SetAspectRatio(InstanceID, AspectRatio)
X_Move360View(InstanceID, LatitudeOffset, LongitudeOffset)
X_Zoom360View(InstanceID, ScaleFactorOffset)
X_Origin360View(InstanceID)
X_ControlCaption(InstanceID, Operation, Name, ResourceURI, CaptionURI, CaptionType, Language, Encoding)
X_GetCaptionState(InstanceID)
	Returns: (Captions, EnabledCaptions)
X_GetServiceCapabilities(InstanceID)
	Returns: (ServiceCapabilities)
X_SetZoom(InstanceID, x, y, w, h)
X_GetTVSlideShow(InstanceID)
	Returns: (CurrentShowState, CurrentThemeId, TotalThemeNumber)
X_SetTVSlideShow(InstanceID, CurrentShowState, CurrentShowTheme)


In [794]:
get_statevar_desc(scpd_root, 'getvolume', 'channel')

{'dataType': 'string', 'allowedValueList': 'Master'}

In [795]:
def print_action_arg_desc(action_name):
    args = get_action_arguments(scpd_root, action_name)
    assert args != None, 'There was no (%s) action found.'%(action_name)
    
    for arg in args:
        print(arg.text)
        desc = get_statevar_desc(scpd_root, action_name, arg.text)
        for k, v in desc.items():
            if k != '':
                print('\t%s: %s'%(k, v))
            
print_action_arg_desc('getvolume')

InstanceID
	dataType: ui4
Channel
	dataType: string
	allowedValueList: Master
CurrentVolume
	dataType: ui2
	allowedValueRange: 
        


# SOAP Call

In [857]:
def create_soap_body(action_name, service_type, soap_encoding, encoding='utf-8'):
    NS_SOAP_ENV = 'http://schemas.xmlsoap.org/soap/envelope'
    print(service_type)
    soap_env = '{%s}'%NS_SOAP_ENV
    m = '{%s}'%service_type
    root = etree.Element(soap_env+'Envelope', nsmap={'SOAP-ENV': NS_SOAP_ENV})
    root.attrib[soap_env+'encodingStyle'] = soap_encoding
    body = etree.SubElement(root, soap_env+'Body')

    action = etree.SubElement(body, m+action_name, nsmap={'m': service_type})
    for key, value in arg_in.items():
        etree.SubElement(action, key).text = str(value)
            
    body = etree.tostring(root, encoding=encoding, xml_declaration=True)
    
    return body

def soap_call(action_name, ctrl_url, service_type, inputs_dict=None):
    soap_encoding = "http://schemas.xmlsoap.org/soap/encoding/"
    soap_env = "http://schemas.xmlsoap.org/soap/envelope"
    
    if inputs_dict == None:
        inputs_dict = {}
        
    soap_body = create_soap_body(action_name, service_type, soap_encoding)
    #soap_action = 'urn:schemas-upnp-org:service:RenderingControl:1#GetVolume'
    soap_action = "%s#%s"%(service_type, action_name)
    #print('SOAP Action: ', soap_action)
    
    headers = {
        'SOAPAction': '"%s"'%(soap_action),
        'Host': '192.168.1.14:9197',
        'Content-Type': 'text/xml',
        'Content-Length': str(len(soap_body)),
    }

    response = requests.post(ctrl_url, soap_body, headers=headers)

    return response.content.strip()

In [858]:
argument_inputs = {'InstanceID': 0, 'Channel': 'Master'}
response = soap_call('GetVolume', ctrl_url, service_type, argument_inputs)
response

urn:schemas-upnp-org:service:RenderingControl:1
{urn:schemas-upnp-org:service:RenderingControl:1} GetVolume


b'<?xml version="1.0" encoding="utf-8"?><s:Envelope s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/" xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><u:GetVolumeResponse xmlns:u="urn:schemas-upnp-org:service:RenderingControl:1"><CurrentVolume>60</CurrentVolume>\r\n</u:GetVolumeResponse></s:Body></s:Envelope>'

In [735]:
def get_scpd_simple(obj, results=None, filter_txt='', filter_tag='', filter_prt_tag='', exact=False):
    if results == None:
        results = []
        
    for v in obj.getchildren():
        v_str = v.text.strip()
        t_str = v.tag.split('}')[-1] if '}' in v.tag else v.tag
        p_str = v.getparent().tag.split('}')[-1] if '}' in v.getparent().tag else v.getparent().tag
        if exact: # Filter is exact match
            if filter_txt.lower() == v_str.lower() and filter_tag.lower() == t_str.lower() and filter_prt_tag.lower() == p_str.lower():
                results += [v]
        else: # Filter Contains key word
            if filter_txt.lower() in v_str.lower() and filter_tag.lower() in t_str.lower() and filter_prt_tag.lower() in p_str.lower():
                results += [v]
        
        get_scpd_simple(v, results=results, filter_txt=filter_txt, filter_tag=filter_tag, filter_prt_tag=filter_prt_tag, exact=exact)
        
    return results

def get_xml_obj(obj, results=None, filter_txt='', filter_tag=''):
    if results == None:
        results = []
        
    for v in obj.getchildren():
        v_str = v.text.strip()
        t_str = v.tag
        if v_str != '' and filter_txt in v_str.lower() and filter_tag in t_str.lower():
            results += [v]
        get_xml_obj(v, results=results, filter_txt=filter_txt, filter_tag=filter_tag)
    return results

In [224]:
def get_xml_simple_from_url(url, filter_txt='', filter_tag=''):
    res = requests.get(url)
    root = etree.fromstring(res.content, base_url=url)
    
    return get_scpd_simple(root, filter_txt=filter_txt, filter_tag=filter_tag)

In [225]:
results = get_xml_simple_from_url(url)
results

[('major', '1'),
 ('minor', '0'),
 ('deviceType', 'urn:schemas-upnp-org:device:MediaRenderer:1'),
 ('X_compatibleId', 'MS_DigitalMediaDeviceClass_DMR_V001'),
 ('X_deviceCategory', 'Display.TV.LCD Multimedia.DMR'),
 ('X_DLNADOC', 'DMR-1.50'),
 ('friendlyName', '[TV] Samsung 6 Series (75)'),
 ('manufacturer', 'Samsung Electronics'),
 ('manufacturerURL', 'http://www.samsung.com/sec'),
 ('modelDescription', 'Samsung TV DMR'),
 ('modelName', 'UN75NU6950'),
 ('modelNumber', 'AllShare1.0'),
 ('modelURL', 'http://www.samsung.com/sec'),
 ('serialNumber', '07ZX3CDN101794W'),
 ('UDN', 'uuid:036cb031-f5f8-4831-bcd7-c0259ee25740'),
 ('mimetype', 'image/jpeg'),
 ('width', '48'),
 ('height', '48'),
 ('depth', '24'),
 ('url', '/icon_SML.jpg'),
 ('mimetype', 'image/jpeg'),
 ('width', '120'),
 ('height', '120'),
 ('depth', '24'),
 ('url', '/icon_LRG.jpg'),
 ('mimetype', 'image/png'),
 ('width', '48'),
 ('height', '48'),
 ('depth', '24'),
 ('url', '/icon_SML.png'),
 ('mimetype', 'image/png'),
 ('width', 

In [226]:
get_xml_simple_from_url(url, filter_tag='control')

[('controlURL', '/upnp/control/RenderingControl1'),
 ('controlURL', '/upnp/control/ConnectionManager1'),
 ('controlURL', '/upnp/control/AVTransport1')]

In [215]:
xml_files = get_xml_simple_from_url(url, filter_txt='.xml')
xml_files

[('SCPDURL', '/RenderingControl_1.xml'),
 ('SCPDURL', '/ConnectionManager_1.xml'),
 ('SCPDURL', '/AVTransport_1.xml')]

In [216]:
xml_url = url + xml_files[0][1]
xml_url

'http://192.168.1.14:9197/dmr/RenderingControl_1.xml'

In [244]:
rendering_control = get_xml_simple_from_url(xml_url)
rendering_control

[('major', '1'),
 ('minor', '0'),
 ('name', 'ListPresets'),
 ('name', 'InstanceID'),
 ('direction', 'in'),
 ('relatedStateVariable', 'A_ARG_TYPE_InstanceID'),
 ('name', 'CurrentPresetNameList'),
 ('direction', 'out'),
 ('relatedStateVariable', 'PresetNameList'),
 ('name', 'SelectPreset'),
 ('name', 'InstanceID'),
 ('direction', 'in'),
 ('relatedStateVariable', 'A_ARG_TYPE_InstanceID'),
 ('name', 'PresetName'),
 ('direction', 'in'),
 ('relatedStateVariable', 'A_ARG_TYPE_PresetName'),
 ('name', 'GetMute'),
 ('name', 'InstanceID'),
 ('direction', 'in'),
 ('relatedStateVariable', 'A_ARG_TYPE_InstanceID'),
 ('name', 'Channel'),
 ('direction', 'in'),
 ('relatedStateVariable', 'A_ARG_TYPE_Channel'),
 ('name', 'CurrentMute'),
 ('direction', 'out'),
 ('relatedStateVariable', 'Mute'),
 ('name', 'SetMute'),
 ('name', 'InstanceID'),
 ('direction', 'in'),
 ('relatedStateVariable', 'A_ARG_TYPE_InstanceID'),
 ('name', 'Channel'),
 ('direction', 'in'),
 ('relatedStateVariable', 'A_ARG_TYPE_Channel'),


In [269]:
"""
    parent: parent tag
    child: child tag
    
    returns child's text
"""
def searchtags_forparents(parent, child):
    res = requests.get(xml_url)
    root = etree.fromstring(res.content, base_url=xml_url)
    tags = get_xml_obj(root, filter_tag=child)
    
    results = []
    for t in tags:
        if parent in t.getparent().tag:
            results += [t.text]
    
    return results

In [270]:
searchtags_forparents('action', 'name')

['ListPresets',
 'SelectPreset',
 'GetMute',
 'SetMute',
 'GetVolume',
 'SetVolume',
 'X_GetAspectRatio',
 'X_SetAspectRatio',
 'X_Move360View',
 'X_Zoom360View',
 'X_Origin360View',
 'X_ControlCaption',
 'X_GetCaptionState',
 'X_GetServiceCapabilities',
 'X_SetZoom',
 'X_GetTVSlideShow',
 'X_SetTVSlideShow']

In [241]:
for v in actions:
    names = get_xml_obj(v, filter_tag='name')
    for n in names:
        print(n.text)

TODO:
- Try GetMute and SetMute


In [202]:
results = get_xml_simple(root, filter_tag='device')
results

[('deviceType', 'urn:schemas-upnp-org:device:MediaRenderer:1'),
 ('X_deviceCategory', 'Display.TV.LCD Multimedia.DMR')]